In [2]:
# Importing necessary libraries 
import random
import numpy as np
import pandas as pd 
from IPython.display import display, HTML

## Question 4

In [2]:
## We run a simulation of the Mexican standoff a large number of times and 
## record the results to confirm if the formula is correct
class Simulation():
    def __init__(self, probs_dict):
        self.probs_dict = probs_dict
        
    def hit(self, p):
        
        return True if random.random() < p else False
    
    def init_conditions(self): # Initial conditions
        self.target = {"Good": "Bad", "Bad": "Good", "Ugly": "Good"}
        self.alive = ["Good", "Bad", "Ugly"]
        
    def update_targets(self): # Updates who tries to kill who after each shoot
        
        if self.alive == ["Bad", "Ugly"]:
            self.target["Bad"] = "Ugly"
            self.target["Ugly"] = "Bad"
        
        elif self.alive == ["Good", "Ugly"]:
            self.target["Good"] = "Ugly"
    
        
    def shoot_once(self): # All the living players shoot once
        
        deads = set()
        for c in self.alive:
            if self.hit(self.probs_dict[c]):
                deads.add(self.target[c])
        
        for dead in deads:
            self.alive.remove(dead)
                    
    def run(self, iterations=100):
        self.results = {"Good":0, "Bad": 0, "Ugly": 0}
        
        for i in range(iterations): # Run for given number of iterations
            
            self.init_conditions()
    
            while len(self.alive) > 1: # While atleast 2 ppl are alive
                self.shoot_once()
                self.update_targets()
                
            
            if len(self.alive) != 0:
                self.results[self.alive[0]] += 1 # Add 1 to whichever player survived
        
        for c, v in self.results.items():
            self.results[c] = v / iterations
            
        return self.results

In [3]:
## Calculates actual mathematical values ##
## Formulae used in the function below are provided in the handwritten solutions ##
def actual_survival_probs(probs_dict):
    x = probs_dict["Good"]
    y = probs_dict["Bad"]
    z = probs_dict["Ugly"]
    
    p_good = x**2 * (1-z)**2 * (1-y) / ((1- (1-x)*(1-y)*(1-z)) * (1 - (1-x)*(1-z)))
    p_bad = (1-x)*y*(1-z)/(1- (1-x)*(1-y)*(1-z))
    p_ugly = 1/(1- (1-x)*(1-y)*(1-z)) * ((1-x)*(1-y)*z + \
                                            x*(1-x)*(1-y)*z*(1-z) + \
                                            x*(1-(1-y)*(1-z)))
    
    return {"Good": p_good, "Bad": p_bad, "Ugly": p_ugly}

In [4]:
probs = {"Good": 0.9, "Bad": 0.75, "Ugly": 0.5}

## 1. Actual probabilities using derived expressions ##
actual = actual_survival_probs(probs)

## 2. Running the simulation to check the probabilities ##
sim = Simulation(probs)
simulated = sim.run(iterations=100000)

d = {"Actual": actual, "Simulated": simulated}
df = pd.DataFrame(d)
df["Percent Error"] = abs(df["Actual"] - df["Simulated"]) / df["Actual"] * 100
print("-"*100)
print("Actual vs Simulated survival probabilities")
print("-"*100)
df.head()

----------------------------------------------------------------------------------------------------
Actual vs Simulated survival probabilities
----------------------------------------------------------------------------------------------------


,Actual,Simulated,Percent Error
Good,0.053964,0.05474,1.437951
Bad,0.037975,0.03895,2.568333
Ugly,0.815823,0.81352,0.282265


## Question 2

In [18]:
## Takes a collinearity graph and performs Depth first search to calculate the number of patterns possible ##
class PatternCounter():
    def __init__(self, graph, num_nodes, at_most_visit=1):
        self.graph = graph
        self.num_nodes = num_nodes
        self.at_most_visit = at_most_visit

        
    def validnodes(self, pattern): # checks which all nodes are visitable from the given node
        
        if pattern == []:
            return list([str(i) for i in range(1, self.num_nodes + 1)])
        
        valid_nodes = []
        last_node = pattern[-1] 
        
        for node in list(map(str,range(1, self.num_nodes+1))):
            if (pattern.count(node) < self.at_most_visit and last_node != node):
                
                overlapped = None
                for t in self.graph[last_node]:
                    if t[0]==node:
                        overlapped = t[1]
                        break
        
                if overlapped is None or overlapped in pattern:
                    valid_nodes.append(node)
            
        
        return valid_nodes
        
    def count_patterns(self, length, pattern): # Runs a recursive function to count patterns
        if length == 0:
            return 1
        
        count = 0
        for node in self.validnodes(pattern):
            count += self.count_patterns(length -1, pattern + [node])
        
        return count
        

In [19]:
## Graphs for different patterns ##

mobilegrid = {
                 "1": [("3","2"), ("7","4"), ("9","5")],
                 "2": [("8","5")],
                 "3": [("1","2"), ("7","5"), ("9","6")],
                 "4": [("6","5")],
                 "5": [],
                 "6": [("4","5")],
                 "7": [("1","4"), ("3","5"), ("9","8")],
                 "8": [("2","5")],
                 "9": [("1","5"), ("3","6"), ("7","8")]
            }

graph_a = {
                 "1": [("3","2")],
                 "2": [("4","3")],
                 "3": [("1","2"), ("7","6"), ("5","4")],
                 "4": [],
                 "5": [("3","4")],
                 "6": [],
                 "7": [("3","6")],
                 "8": [],
                 "9": []
                }

graph_b ={
            "1":[],
            "2":[],
            "3":[],
            "4":[],
            "5":[],
            "6":[],
            "7":[],
            "8":[],
            "9":[] 
         }


graph_c ={
            "1": [("3","2"),("4","2"),("5","2")],
            "2": [("4","3"),("5","3")],
            "3": [("5","4"),("7","6"),("9","8")],
            "4": [("2","3"),("1","3"),("1","2")],
            "5": [("3","4"),("2","4"),("1","4")],
            "6": [("8","3"),("9","3")],
            "7": [("3","6")],
            "8": [("6","3")],
            "9": [("3","8")]
        }

In [20]:
## Display results for the given patterns ##
graphs = {"Mobile Grid": mobilegrid, "Pattern A": graph_a, "Pattern B": graph_b, "Pattern C": graph_c}
to_visit_at_most = [1, 2]

for i in to_visit_at_most:
    
    d = {}
    for name, graph in graphs.items():
        
        d[name] = []
        counter = PatternCounter(graph, num_nodes=9, at_most_visit=i)
        for n in range(4, 10):
            d[name].append(counter.count_patterns(n, []))
            
    print("-"*100)
    print(f"Case {i}: A node can be visited atmost {i} time(s)")
    print("-"*100)
    df = pd.DataFrame(d, index= [str(i) for i in range(4, 10)]).T
    df["Total possibilities"] = df.sum(axis=1)
    display(HTML(df.to_html()))      

----------------------------------------------------------------------------------------------------
Case 1: A node can be visited atmost 1 time(s)
----------------------------------------------------------------------------------------------------


,4,5,6,7,8,9,Total possibilities
Mobile Grid,1624,7152,26016,72912,140704,140704,389112
Pattern A,2330,10898,41288,119160,233520,233520,640716
Pattern B,3024,15120,60480,181440,362880,362880,985824
Pattern C,1439,6078,21526,59779,115154,115154,319130


----------------------------------------------------------------------------------------------------
Case 2: A node can be visited atmost 2 time(s)
----------------------------------------------------------------------------------------------------


,4,5,6,7,8,9,Total possibilities
Mobile Grid,2568,17792,122208,824304,5390304,33769696,40126872
Pattern A,3608,26730,193607,1357393,9127059,58288219,68996616
Pattern B,4608,36288,277704,2044224,14336784,94847760,111547368
Pattern C,2275,14975,98094,633136,3993376,24351209,29093065


### Smudge attack

In a smudge attack a person tries to identify the pattern of the mobile based on the smudges left over the screen  by the user after entering the pattern a lote of times. In patterns where a lot of points are collinear, smudges tend to be like straight lines. 

Imagine 3 collinear points A-B-C. If a smudge is present on the straight line it is only possible that the combination A-B-C or C-B-A is a part of the pattern. 

In case of 3 non-collinear points, there will be a triangular smudge over that area leaving us with a total of 3! = 6 combinations which can be a part of the pattern. Hence, patterns with more collinearity are more prone to smudge attacks.

For vulnerability to smudge attacks, Pattern C > Pattern A > Pattern B

## Question 3

In [3]:
# Reading the dataset
df_train = pd.read_csv(r"Cancer_train.csv", index_col=0)
df_test = pd.read_csv(r"Cancer_test.csv", index_col=0)

In [4]:
df_train.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,91813702,B,12.340,12.27,78.94,468.5,0.09003,0.06307,0.02958,0.026470,...,13.610,19.27,87.22,564.9,0.1292,0.2074,0.1791,0.10700,0.3110,0.07592
1,854039,M,16.130,17.88,107.00,807.2,0.10400,0.15590,0.13540,0.077520,...,20.210,27.26,132.70,1261.0,0.1446,0.5804,0.5274,0.18640,0.4270,0.12330
2,858477,B,8.618,11.79,54.34,224.5,0.09752,0.05272,0.02061,0.007799,...,9.507,15.40,59.90,274.9,0.1733,0.1239,0.1168,0.04419,0.3220,0.09026
3,846381,M,15.850,23.95,103.70,782.7,0.08401,0.10020,0.09938,0.053640,...,16.840,27.66,112.00,876.5,0.1131,0.1924,0.2322,0.11190,0.2809,0.06287
4,852973,M,15.300,25.27,102.40,732.4,0.10820,0.16970,0.16830,0.087510,...,20.270,36.71,149.30,1269.0,0.1641,0.6110,0.6335,0.20240,0.4027,0.09876


In [5]:
print(df_train.keys())

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')


In [19]:
class BayesClassifier(): # Bayes classifier
    def __init__(self):
        pass
    
    def fit(self, X_train, y_train): # Calculates the mean and standard deviation for each feature for both classes
        self.stats = {0: {}, 1: {}}
        
        # Class 0 stats 
        arr0 = X_train[y_train==0].to_numpy()
        self.stats[0]["mean"] = np.mean(arr0, axis=0)
        self.stats[0]["std"] = np.std(arr0, axis=0)
        self.stats[0]["Prior"] = np.log(0.6244979919678715)
        
        # Class 1 stats 
        arr1 = X_train[y_train==1].to_numpy()
        self.stats[1]["mean"] = np.mean(arr1, axis=0)
        self.stats[1]["std"] = np.std(arr1, axis=0)
        self.stats[1]["Prior"] = np.log(0.3755020080321285)
    
    
    def find_prob_sum(self, x, mu, sigma): 
        # Calculates sum of log of featurewise probabilites instead of multiplying all featurewise probabilities
        return np.sum(- np.log(sigma) - (x - mu)**2 / (2 * sigma**2))
    
    def predict(self, X_test): 
        # Calculates probabilities for each class per sample and assigns the class with maximum probability
        
        result = []
        X_test = X_test.to_numpy()
        
        for i in range(X_test.shape[0]):
            
            sample = X_test[i, :]
            logp0 = self.find_prob_sum(sample, self.stats[0]["mean"], self.stats[0]["std"])  + self.stats[0]["Prior"]
            logp1 = self.find_prob_sum(sample, self.stats[1]["mean"], self.stats[1]["std"])  + self.stats[1]["Prior"]
            print('logp0:',logp0)
            print('logp1:',logp1)
            if logp0 > logp1:
                result.append(0)
            else:
                result.append(1)
                
                
        return np.array(result)
    
    def get_accuracy(self, X_test, y_test): # Calculates accuracy

        preds = self.predict(X_test)        
        return np.sum(preds == y_test) / len(y_test)
                

def preprocess_data(data_path, cols=[]): # preprocesses the dataset and gives 0/1 labels
    
    df = pd.read_csv(data_path, index_col=0)
    df.drop(columns=["id"], inplace=True)
    
    if cols != []:
        cols.append("diagnosis")
        df = df[cols]
    
    df.replace({"diagnosis": {"B": 0, "M": 1}}, inplace=True)
    y = df.pop("diagnosis").to_numpy()
    
    return df, y

In [20]:
## 1. Train the model with only 2 features ##

# Prepare the dataset #
X_train, y_train = preprocess_data(r"Cancer_train.csv", cols=["concave points_worst", "radius_mean"])
X_test, y_test = preprocess_data(r"Cancer_test.csv", cols=["concave points_worst", "radius_mean"])

# Model training and testing #
clf = BayesClassifier()
clf.fit(X_train, y_train)

acc = clf.get_accuracy(X_test, y_test)
print(f"Model accuracy with concave_points_worst and radius_mean: {acc*100: .4f} %")


## 2. Train the model with all features ##

# Prepare the dataset #
X_train, y_train = preprocess_data(r"Cancer_train.csv")
X_test, y_test = preprocess_data(r"Cancer_test.csv")

# Model training and testing #
clf = BayesClassifier()
clf.fit(X_train, y_train)

acc = clf.get_accuracy(X_test, y_test)
print(f"Model accuracy with all features: {acc*100:.4f} %")

logp0: 0.8389134916731571
logp1: -2.536434630992238
logp0: 2.1353601082130167
logp1: -3.1226007938884863
logp0: 0.5091768787470357
logp1: -0.2995061553399331
logp0: -49.4614583052954
logp1: -6.110461853520091
logp0: -27.887273366523942
logp1: -1.5723102668028353
logp0: 1.9963441874318812
logp1: -2.6068468467162207
logp0: 1.5134392106774066
logp1: -1.1945120377246297
logp0: -49.494831834963044
logp1: -6.066553947949413
logp0: 1.5689959496569683
logp1: -6.172798709307599
logp0: -3.2762587350174956
logp1: 0.7018495580827718
logp0: 0.5091692458030297
logp1: -0.29414561196943756
logp0: 1.8873341025939636
logp1: -4.1731422635695665
logp0: 0.2756154009221659
logp1: -7.259993276038508
logp0: 1.959453378530288
logp1: -1.7114633880129624
logp0: 1.861645519808771
logp1: -1.9065073441650267
logp0: 0.018208283247021895
logp1: -4.452464616507019
logp0: 2.294346496405365
logp1: -3.5212787130105276
logp0: 1.5318378299367188
logp1: -5.642815030682509
logp0: -0.38678517621663894
logp1: 0.177547898915918